In [2]:
from SinsIIStatsThing import getSinsData, createWeaponDict
import json, pprint, glob

In [3]:
with open(r'Patch 42.05 Localized Text\en.localized_text', 'r') as file:
    LOCALIZED_TEXT = json.load(file)

subjectGlob = glob.glob(r'Patch 42.05 Entities\**.research_subject')

# with open(r'Patch 41.06 Uniforms\planet.uniforms', 'r') as file:
#     planetUniform = json.load(file)

In [13]:
subjectDict = {}

for i in subjectGlob:
    with open(i, 'r') as file:
        subject = json.load(file)

        nameCall = subject.pop('name')
        name = LOCALIZED_TEXT.get(nameCall)
        subject['name'] = name
        if name == None:
            print(f"{i} has no name")
            name = i.split("\\")[-1].replace(".research_subject", "")


        try:
            subject['description'] = LOCALIZED_TEXT.get(subject['description'], None)
        except:
            pass

        # if 'vasari' in nameCall:
        #     name = 'Vasari ' + name
        # elif 'trader' in nameCall:
        #     name = 'TEC ' + name
        # elif 'advent' in nameCall:
        #     name = 'Advent ' + name

        subjectDict[name] = subject

pprint.pprint(subjectDict)

Patch 42.05 Entities\trader_shield_burst_restore.research_subject has no name
{'Ablative Armor Unlock': {'domain': 'military',
                           'field': 'military_engineering',
                           'field_coord': [5, 1],
                           'hud_icon': 'trader_beam_armor_unit_item_hud_icon',
                           'name': 'Ablative Armor Unlock',
                           'name_uppercase': 'trader_unlock_beam_armor_unit_item_research_subject_name_uppercase',
                           'prerequisites': [['trader_hull_damage_reduction_1']],
                           'price': {'credits': 1250.0,
                                     'crystal': 350.0,
                                     'metal': 225.0},
                           'research_time': 360.0,
                           'tier': 2,
                           'tooltip_picture': 'trader_beam_armor_unit_item_tooltip_picture',
                           'version': 0},
 'Acclimatization of Will': {'arbitary

In [11]:
fields = []
for _, i in subjectDict.items():
    if i['field'] not in fields:
        fields.append(i['field'])

for j in fields:
    print(j)

military_ascension
civilian_temp
civilian_material
civilian_evangelism
military_energy
military_protection
military_temp
civilian_reclamation
civilian_nanotech
civilian_industry
military_experimental
civilian_phase_mastery
military_remnants
civilian_annexation
military_engineering
military_armament
military_mobilization
civilian_policy
military_survival
military_revival
civilian_oppression
military_assault


In [20]:
for i in subjectDict.keys():
    print(f"[[File:SoaSE2_{i.replace(" ", "_")}.png|thumb|{i}]]") #

[[File:SoaSE2_Ritualized_Instruction.png|thumb|Ritualized Instruction]]
[[File:SoaSE2_Ascension_Rites.png|thumb|Ascension Rites]]
[[File:SoaSE2_Reanimation.png|thumb|Reanimation]]
[[File:SoaSE2_Commerce_Integration.png|thumb|Commerce Integration]]
[[File:SoaSE2_Commerce_Adaptation.png|thumb|Commerce Adaptation]]
[[File:SoaSE2_Commerce_Acceptance.png|thumb|Commerce Acceptance]]
[[File:SoaSE2_Forewarning.png|thumb|Forewarning]]
[[File:SoaSE2_Acute_Premonitions.png|thumb|Acute Premonitions]]
[[File:SoaSE2_Andvar_Cultivation.png|thumb|Andvar Cultivation]]
[[File:SoaSE2_Wall_of_Faith.png|thumb|Wall of Faith]]
[[File:SoaSE2_Fortress_of_Faith.png|thumb|Fortress of Faith]]
[[File:SoaSE2_High_Efficiency_Jumps.png|thumb|High Efficiency Jumps]]
[[File:SoaSE2_Improved_Meteoroid_Control.png|thumb|Improved Meteoroid Control]]
[[File:SoaSE2_Acclimatization_of_Will.png|thumb|Acclimatization of Will]]
[[File:SoaSE2_Clarity_of_Purpose.png|thumb|Clarity of Purpose]]
[[File:SoaSE2_Global_Unity.png|thumb|G